In [1]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import warnings
import tika
from tika import parser
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
import spacy
import re
from nltk.corpus import stopwords
import numpy as np
import pandas as pd
warnings.filterwarnings('ignore')
import multiprocessing
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from sklearn import utils
from sklearn.model_selection import train_test_split
import gensim
from sklearn.linear_model import LogisticRegression
import seaborn as sns
import matplotlib.pyplot as plt


In [2]:
base = ['data/AS_', 'data/Ml_', 'data/Bd_', 'data/Hd_']
dados = []
num_data = []
for elem in range(0, len(base)):
    count = 0
    if elem != 0:
        final = 10
    else:
        final = 34
        
    for i in range(1, final):
        terminal = base[elem] + str(i) + '.pdf'
        dados.append(terminal)
        count+=1
    
    num_data.append(count)



In [3]:
base = ['data/Ml_', 'data/Bd_', 'data/Hd_', 'data/AS_']
dados = []
num_data = []
for elem in range(0, len(base)):
    count = 0
    final = 10    
    for i in range(1, final):
        terminal = base[elem] + str(i) + '.pdf'
        dados.append(terminal)
        count+=1
    
    num_data.append(count)



In [4]:
print(dados)

['data/Ml_1.pdf', 'data/Ml_2.pdf', 'data/Ml_3.pdf', 'data/Ml_4.pdf', 'data/Ml_5.pdf', 'data/Ml_6.pdf', 'data/Ml_7.pdf', 'data/Ml_8.pdf', 'data/Ml_9.pdf', 'data/Bd_1.pdf', 'data/Bd_2.pdf', 'data/Bd_3.pdf', 'data/Bd_4.pdf', 'data/Bd_5.pdf', 'data/Bd_6.pdf', 'data/Bd_7.pdf', 'data/Bd_8.pdf', 'data/Bd_9.pdf', 'data/Hd_1.pdf', 'data/Hd_2.pdf', 'data/Hd_3.pdf', 'data/Hd_4.pdf', 'data/Hd_5.pdf', 'data/Hd_6.pdf', 'data/Hd_7.pdf', 'data/Hd_8.pdf', 'data/Hd_9.pdf', 'data/AS_1.pdf', 'data/AS_2.pdf', 'data/AS_3.pdf', 'data/AS_4.pdf', 'data/AS_5.pdf', 'data/AS_6.pdf', 'data/AS_7.pdf', 'data/AS_8.pdf', 'data/AS_9.pdf']


In [5]:
# data = ['data/Ml_1.pdf', 'data/Ml_2.pdf', 'data/Ml_3.pdf', 'data/Ml_4.pdf', 'data/Ml_5.pdf', 'data/Ml_6.pdf', 'data/Ml_7.pdf']
biblioteca = []
tika.initVM()
for i in range(0, len(dados)):
    
    parsed = parser.from_file(dados[i])
    metadados = parsed["metadata"]
    texto = parsed["content"]
    biblioteca.append(texto)

In [6]:
print(len(biblioteca))
# print(len(data))

36


In [7]:
# f = open ("artigo2.txt", "w")
# f.write (str(texto))
# f.close ()

In [8]:

# f = open ("artigo2.txt", "r")
# texto = f.read()

In [9]:
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [10]:
def split_pdf(string): 
    li = list(string.split(" ")) 
    return li

In [11]:
def first_clear(data):
    # Remove Emails
    data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

    #remove links and mark
    data = [re.sub('(https:\S+)|(@)|(¿)', '', sent) for sent in data]
    
    data = [re.sub('(http:\S+)|(@)|(¿)', '', sent) for sent in data]
    
    data = [re.sub('(www.:\S+)|(@)|(¿)', '', sent) for sent in data]

    # Remove new line characters
    data = [re.sub('\s+', ' ', sent) for sent in data]

    # remove spaces in begining and end
    data = [re.sub(' +', ' ', sent) for sent in data]

    # Remove distracting single quotes
    data = [re.sub("\'", "", sent) for sent in data]

    return data

In [12]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

In [13]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

In [14]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [15]:
def convert(s): 
    a = 0
    # initialization of string to "" 
    new = "" 
  
    # traverse in the string  
    for x in s: 
        new += x + ' '
        a+=1
  
    # return string  
    return new, a

In [16]:
data_cleared = []

In [17]:
total_char = 0
for i in range(0, len(biblioteca)):
    texto = biblioteca[i]
    data = split_pdf(texto)
    data = first_clear(data)
    d, a = convert(data)
    
    total_char += a
    data_cleared.append(d)


In [18]:
print(len(data_cleared))
print(total_char)

36
292303


In [19]:
df = pd.DataFrame(data_cleared)
df = df.rename(columns={0: "text"})

In [20]:
df.head()

,text
0,A Comparative Study on Feature Selection in T...
1,fqs003 183..196 Detecting authorship deceptio...
2,New Machine Learning Methods Demonstrate the ...
3,fqq001 215..224 A comparative study of machin...
4,fqi039 259..274 A New Approach to the Study o...


In [21]:
l = []
for i in range(0, len(dados)):
    if 'AS' in str(dados[i]):
        l.append('Analyze_Statistics')
    elif 'Ml' in str(dados[i]):
        l.append('Machine_Learning')
    elif 'Bd' in str(dados[i]):
        l.append('Big_Data')
    else:
        l.append('Digital_Humanities')

In [22]:
df['classe'] = l

In [23]:
df.head()

,text,classe
0,A Comparative Study on Feature Selection in T...,Machine_Learning
1,fqs003 183..196 Detecting authorship deceptio...,Machine_Learning
2,New Machine Learning Methods Demonstrate the ...,Machine_Learning
3,fqq001 215..224 A comparative study of machin...,Machine_Learning
4,fqi039 259..274 A New Approach to the Study o...,Machine_Learning


In [24]:
train, test = train_test_split(df, test_size=0.2, random_state=42)


In [25]:
import nltk
from nltk.corpus import stopwords
def tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            if len(word) < 2:
                continue
            tokens.append(word.lower())
    return tokens

In [26]:
train_tagged = train.apply(
    lambda r: TaggedDocument(words=tokenize_text(r['text']), tags=[r.classe]), axis=1)
test_tagged = test.apply(
    lambda r: TaggedDocument(words=tokenize_text(r['text']), tags=[r.classe]), axis=1)

In [27]:
train_tagged


8     ([acmcs00.tex, machine, learning, in, automate...
17    ([foundations, and, trends, in, information, r...
9     ([unified, approach, to, mapping, and, cluster...
34    ([microsoft, word, dabagh.doc, metodološki, zv...
0     ([comparative, study, on, feature, selection, ...
4     ([fqi039, 259..274, new, approach, to, the, st...
29    ([fqi063, 311..326, tool, for, literary, studi...
15    ([doi, 10.1126/science.1199644, 176, 2011, 331...
19    ([9-25_1, golumbia, 156-76, .indd, volume, 25,...
5     ([using, markov, chains, for, identification, ...
11    ([fqs010, 139..154, automatic, prediction, of,...
1     ([fqs003, 183..196, detecting, authorship, dec...
24    ([8-25_1, lennon, 132-55, .indd, volume, 25, n...
2     ([new, machine, learning, methods, demonstrate...
33    ([journal, of, machine, learning, research, 20...
3     ([fqq001, 215..224, comparative, study, of, ma...
32    ([pone.0087908, 1..10, computational, approach...
23    ([signs, symbols, and, discourses, new, di

In [28]:
import multiprocessing

cores = multiprocessing.cpu_count()

In [29]:
model_dbow = Doc2Vec(dm=1, vector_size=500, negative=5, hs=0, min_count=2, sample = 0, workers=cores)
model_dbow.build_vocab([x for x in tqdm(train_tagged.values)])

100%|██████████| 28/28 [00:00<00:00, 45661.16it/s]


In [30]:
%%time
for epoch in range(30):
    model_dbow.train(utils.shuffle([x for x in tqdm(train_tagged.values)]), total_examples=len(train_tagged.values), epochs=1)
    model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha

100%|██████████| 28/28 [00:00<00:00, 228928.87it/s]


CPU times: user 31.1 s, sys: 383 ms, total: 31.5 s
Wall time: 13 s


In [31]:
def vec_for_learning(model, tagged_docs):
    sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words, steps=20)) for doc in sents])
    return targets, regressors

In [32]:

y_train, X_train = vec_for_learning(model_dbow, train_tagged)
y_test, X_test = vec_for_learning(model_dbow, test_tagged)

In [33]:
logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

In [34]:
from sklearn.metrics import accuracy_score, f1_score

print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))

Testing accuracy 0.5
Testing F1 score: 0.55
